In [ ]:
# Hello Friends!

# Let us Load essential libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:

# Problem Description:
#    1. The median housing prices of various towns inside Boston is to be predicted
#       by using the given predictor variables.
       


In [ ]:
#%% Read the data and look at colums
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
print ("Train file...")
train.head()

In [ ]:
print ("\nTestfile...")
test.head()

In [ ]:
# lets find missing values

train.info()

In [ ]:
# Hurrah! no missing values. Doesnt mean no incorrect values though!
# What about test data
test.info()

In [ ]:
# Lets just have a look at basic data properties
train.describe()

In [ ]:
# Looks like all are continuous variables apart from chas.
# Want to make sure ID  represents the town id. Has to as it not defined in the Kaggle dataset description.Still!

In [ ]:
# If Id values in train are all unqiue
print ("Are all values in ID in train file unique? %s"% (
                                len(train.ID.unique()) == len(train)))
  
# If Id values in test are all unqiue
print ("Are all values in ID in test file unique? %s"% (
                                len(test.ID.unique()) == len(test)))

# if No Id values in train present in test?

trainintest = [id1 for id1 in train.ID.values if id1 in test.ID.values ]
print ("No of items of ID in train in test? %s"% (
                                len(trainintest)))


testintrain = [id1 for id1 in test.ID.values if id1 in train.ID.values ]
print ("No of items of ID in test in train? %s"% (
                                len(testintrain)))


In [ ]:
# So, all values in ID are unqiue and we are 100% certain that the represent
# town id.  Since this is just an identifier and of no use in prediction, 
# we will drop it.

train.drop('ID', axis = 1, inplace = 1)

In [ ]:
# I dont understand the black variable. The formula 1000(Bk - 0.63)^2 is too complicated to intitutively understand it.
# Hence decoding it.

trainblack = pd.DataFrame({})
trainblack['given'] = train['black']
trainblack['decoded'] = 0.63 - np.sqrt(trainblack/1000) # assumes max black ratio is 0.63
sns.regplot(x = "given", y = "decoded", fit_reg = False, data = trainblack)



In [ ]:
# We will assume inverse linear relationship.
# So a Low black value in train means high black population and vice versa
del trainblack

In [ ]:
# functiont to plot scatterplots of all independent variables with dependent variables

def scattermatrix(df):

    f, ax1= plt.subplots(4,4, sharey = True,figsize = (20,20) )

    count = 0
    for i in np.arange(4):
        for j in np.arange(4):
            try: 
                sns.set_context(font_scale = 10)
                sns.regplot(df[df.columns[count]], df.medv, ax = ax1[i,j])
                count = count + 1
            except IndexError:
                pass
    plt.subplots_adjust(left = 0.01, right = .99, bottom = 0.12, top = 0.90,
                                wspace = .10, hspace = .12)  


    return 

In [ ]:
# Lets check the scatterplot of all independent variables with dependent variables
scattermatrix (train)

In [ ]:
# Hang on. We will look at relationships. But there is something intriguing which i want to look at before.
# Look at the vertical bars in rad, tax, ptratio, zn, indus. They seem related. Lets explore it.

In [ ]:
# Easy way to extract the data is rad> 20

train_intriguing = train[train.rad > 15]
fig2= scattermatrix (train_intriguing)
fig2

In [ ]:
# Ayela! zn, indus, rad, tax, ptratio same to same. Wat Lag gayi mamu!
# Whats happening:
#    1. Same Broad locatlity: No: then dis would be same.
#    2. Data Error! Yes: looks like

# Solutions: ????
#    1. Mark them as missing value and repopulate? 
#    2. If yes, then what if error propagates in test set?
#    3. Ignore them?  Can't. a few like zn, ptratio etc has some predictive powers.

In [ ]:
# Lets focus back on relationships and if we have a problem of multicollinearity

In [ ]:
# I want firt look at the problem of multicolinerity. 
# I have a perfect test in mind. Visual test!!!  

In [ ]:
sns.pairplot(train)

In [ ]:
# I know too messy!
# Some of correlated vatiables I could find are:
# nox, dis
# rm, lstat
# age, dis

# Lets see each of them

In [ ]:
# nox, dis
sns.regplot ("nox", "dis", data = train)

In [ ]:
# rm, lstat
sns.regplot ("rm", "lstat", data = train)

In [ ]:
# age, dis
sns.regplot ("age", "dis", data = train)

In [ ]:
# Think we should drop one  of nox, dis , while retain other two.

In [ ]:

sns.regplot("nox", "medv", data = train)


In [ ]:
sns.regplot("dis", "medv", data = train)


In [ ]:
#  dis is more cattered so will drop it.
train.drop('dis', axis =1, inplace = True)

In [ ]:
# One more graph for your biased, intolerant people!
sns.regplot("crim", "black", data = train)


In [ ]:
# Sorry to diappoint, but absolutely no relationship! Boston is crimefree and also blackfree!

In [ ]:
# Now lets check our scattermatix once again!!
fig = scattermatrix (train)
fig

In [ ]:
# rad and tax look irrelevant. So remove
train.drop(['rad', 'tax'], axis = 1, inplace = 1)

In [ ]:
# lets replot
scattermatrix(train)

In [ ]:
# I think it looks good now!!
predictors = ['crim', 'zn', 'indus', 'chas', 'nox', 'rm', 'age', 'ptratio', 'black','lstat']              

In [ ]:
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor
from sklearn import cross_validation
Y_train = np.asarray(train['medv'], dtype="|S6")

In [ ]:
from sklearn import linear_model
scores = cross_validation.cross_val_score(
    alg,
    train[predictors].values,
    Y_train)
print (scores.mean())

In [ ]:

rf  = RandomForestRegressor()

scores = cross_validation.cross_val_score(
    rf,
    train[predictors].values,
    Y_train)

print (scores.mean())